## decorator
本质上，decorator就是一个返回函数的高阶函数。所以，我们要定义一个能打印日志的decorator，可以定义如下：

如下代码，log 本质上是一个 接受一个函数作为参数，并返回一个函数的高阶函数。

使用 log 装饰 now 后再执行相当于：
now = log(now)
now()

In [6]:
def log(func):
    def wrapper(*args, **kwargs):
        print(f'Calling: {func.__name__}()')
        return func(*args, **kwargs)
    return wrapper

@log
def now():
    print('2025-12-25')

now()

Calling: now()
2025-12-25


也就是说 now 函数被重新指向后 再执行。


## importlib

用来动态导入配置文件

因为根据任务不同，需要导入的配置文件可能不同。使用 importlib 方法可以根据任务的类型来动态的选择需要导入的配置文件

```python
import importlib


task_module = import.import_module(self.task_mapping[config.task_type])
```

## contextlib

python 中读写文件后要正确的关闭。
一个方法是使用 try...finally
```python
try:
    f = open('/path/to/file', 'r')
    f.read()
finally:
    if f:
        f.close()
```
python 提供了 with 语句供我们方便的使用文件资源，并自动关闭，上面的代码可以简化为
```python
with open('/path/to/file', 'r') as f:
    f.read()
```

实际上除了 open 函数返回的 fp(file pointer) 对象可以使用 with 语句。
只要实现了上下文管理就可以使用 with 语句。
上下文管理是通过 ```__enter__``` 和 ```__excit__``` 两个方法实现的。
例如下面的类，实现了这两个方法：

In [1]:
class Query(object):
    def __init__(self, name):
        self.name = name

    def __enter__(self):
        print('Begin')
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        if exc_type:
            print('Error')
        else:
            print('End')
    
    def query(self):
        print('Query info about %s...' % self.name)

with Query('Bob') as q:
    q.query()

Begin
Query info about Bob...
End


编写 ```__enter__``` 和 ```__exit__``` 仍然很繁琐，因此Python的标准库contextlib提供了更简单的写法，上面的代码可以改写如下：

In [5]:
from contextlib import contextmanager

class Query(object):
    def __init__(self, name):
        self.name = name

    def query(self):
        print('Query info about %s...' % self.name)

@contextmanager
def create_query(name):
    print('Begin')
    q = Query(name)
    yield q
    print('End')

# @contextmanager这个decorator接受一个generator，用yield语句把with ... as var把变量输出出去，然后，with语句就可以正常地工作了：

with create_query('Bob') as q:
    q.query()

# 这里 q 在退出 with 后没有释放资源，如果要释放资源，可以在 yield 后或者 __exit__ 中释放资源
q.query()

Begin
Query info about Bob...
End
Query info about Bob...


很多时候，我们希望在某段代码执行前后自动执行特定代码，也可以用@contextmanager实现。例如：

In [3]:
@contextmanager
def tag(name):
    print(name)
    yield
    print('/' + name)

with tag('h1'):
    print('Hello')
    print('world')

h1
Hello
world
/h1


上面的代码首先执行 yield 之前的语句

再执行with 内所有的语句

最后执行 yield 后的代码

简单来说 @contextmanager让我们通过编写generator来简化上下文管理。

如果一个对象没有实现上下文，我们就不能把它用于with语句。这个时候，可以用closing()来把该对象变为上下文对象。例如，用with语句使用urlopen()：
```python
from contextlib import closing
from urllib.request import urlopen

with closing(urlopen('https://www.python.org')) as page:
    for line in page:
        print(line)
```

closing也是一个经过@contextmanager装饰的generator，这个generator编写起来其实非常简单：

```python
@contextmanager
def closing(thing):
    try:
        yield thing
    finally:
        thing.close()
```

它的作用就是把任意对象变为上下文对象，并支持with语句。

@contextlib还有一些其他decorator，便于我们编写更简洁的代码。

## Partial function

通过设定参数的默认值，可以降低函数调用的难度。而偏函数也可以做到这一点。举例如下：

int()函数可以把字符串转换为整数，当仅传入字符串时，int()函数默认按十进制转换：

In [7]:
int('12345')

12345

但int()函数还提供额外的base参数，默认值为10。如果传入base参数，就可以做N进制的转换：

(base = 8, 表示的是输入的参数是 8 进制， 下面的例子中 '12345' 是 8 进制或者 16进制)

In [8]:
print(f"int('12345', base=8) = {int('12345', base=8)}")

print(f"int('12345', 16) = {int('12345', 16)}")

int('12345', base=8) = 5349
int('12345', 16) = 74565


假设要转换大量的二进制字符串，每次都传入int(x, base=2)非常麻烦，于是，我们想到，可以定义一个int2()的函数，默认把base=2传进去：

In [10]:
def int2(x, base=2):
    return int(x, base)

print(f"int2('1000000') = {int2('1000000')}")

print(f"int2('1010101') = {int2('1010101')}")


int2('1000000') = 64
int2('1010101') = 85


```functools.partial``` 就是帮助我们创建一个偏函数的，不需要我们自己定义```int2()```，可以直接使用下面的代码创建一个新的函数int2：

In [13]:
import functools
int2 = functools.partial(int, base=2)

print(f"int2('1000000') = {int2('1000000')}")

print(f"int2('1010101') = {int2('1010101')}")

int2('1000000') = 64
int2('1010101') = 85


所以，简单总结functools.partial的作用就是，把一个函数的某些参数给固定住（也就是设置默认值），返回一个新的函数，调用这个新函数会更简单。

注意到上面的新的int2函数，仅仅是把base参数重新设定默认值为2，但也可以在函数调用时传入其他值：

In [14]:
int2('1000000', base=10)

1000000

最后，创建偏函数时，实际上可以接收函数对象、*args和**kw这3个参数，当传入：
```python
int2 = functools.partial(int, base=2)
```
实际上固定了int()函数的关键字参数base，也就是：
int2('10010')
相当于：
```python
kw = { 'base': 2 }
int('10010', **kw)
```

当传入:
```python
max2 = functools.partial(max, 10)
```

实际上会把10作为*args的一部分自动加到左边，也就是：
```python
max2(5, 6, 7)
```
相当于
```python
args = (10, 5, 6, 7)
max(*args)
```
结果为 10


## 网络编程

IP协议负责把数据从一台计算机通过网络发送到另一台计算机。

IP地址实际上是一个32位整数（称为IPv4），以字符串表示的IP地址如192.168.0.1实际上是把32位整数按8位分组后的数字表示，目的是便于阅读。

IPv6地址实际上是一个128位整数，它是目前使用的IPv4的升级版，以字符串表示类似于2001:0db8:85a3:0042:1000:8a2e:0370:7334。

TCP协议则是建立在IP协议之上的。TCP协议负责在两台计算机之间建立可靠连接，保证数据包按顺序到达。

一个TCP报文除了包含要传输的数据外，还包含源IP地址和目标IP地址，源端口和目标端口。

端口有什么作用？在两台计算机通信时，只发IP地址是不够的，因为同一台计算机上跑着多个网络程序。一个TCP报文来了之后，到底是交给浏览器还是QQ，就需要端口号来区分。每个网络程序都向操作系统申请唯一的端口号，这样，两个进程在两台计算机之间建立网络连接就需要各自的IP地址和各自的端口号。

一个进程也可能同时与多个计算机建立链接，因此它会申请很多端口。



### Socket

Socket是网络编程的一个抽象概念。通常我们用一个Socket表示“打开了一个网络链接”，而打开一个Socket需要知道目标计算机的IP地址和端口号，再指定协议类型即可。

大多数连接都是可靠的TCP连接。创建TCP连接时，主动发起连接的叫客户端，被动响应连接的叫服务器。



## 子进程
### Popen


### Filelock

首先要明白为什么需要加锁，举个例子：n 个读者到图书馆查阅 1 本书籍。但是这本书同一时间只能借给一个人。我们需要如何协调？

Python filelock库是一个用于文件锁定的工具，可以帮助开发者在多线程或多进程环境中管理文件的并发访问，避免数据竞争和冲突。

- 支持基于文件的锁定机制
- 提供了上下文管理器来自动管理锁的获取和释放
- 支持超时机制，防止死锁情况发生
- 可以管理多个文件的锁定状态

#### 获取文件锁

在这个示例中，创建了一个文件锁lock，并使用上下文管理器with lock来获取文件锁，然后在锁定范围内执行操作。

In [8]:
from filelock import FileLock

# 创建文件锁
lock = FileLock("data.txt.lock")

# 获取文件锁
with lock:
    # 在锁定范围内执行操作
    with open("data.txt", "a") as file:
        file.write("Hello, World!\n")

with lock:
    with open("data.txt", "r") as f:  # 打开文件
        data = f.read()  # 读取文件
        print(data)


Hello, World!
Hello, World!
Hello, World!
Hello, World!
Hello, World!
Hello, World!



Hello, World!
Hello, World!
Hello, World!
Hello, World!



#### 超时机制

在这个示例中，创建了一个超时为5秒的文件锁lock，并使用lock.acquire(timeout=2)来尝试获取文件锁，如果超时则会抛出Timeout异常。

In [3]:
from filelock import Timeout, FileLock

# create the lock, and set a 5 second timeout
lock = FileLock("data.txt.lock", timeout=5)

try:
    # try to acquire the lock
    with lock.acquire(timeout=2):
        # execute operation in the locked scope
        with open("data.txt", "a") as file:
            file.write("Hello, World!\n")
except Timeout:
    print("acquire lock timeout!")


#### 文件锁的释放

In [4]:
from filelock import FileLock
 
# 创建文件锁
lock = FileLock("data.txt.lock")
 
# 获取文件锁
with lock:
    # 在锁定范围内执行操作
    with open("data.txt", "a") as file:
        file.write("Hello, World!\n")
    
    # 手动释放文件锁
    lock.release()

In [1]:
## Runner


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset

from mmengine.model import BaseModel
from mmengine.evaluator import BaseMetric
from mmengine.registry import MODELS, DATASETS, METRICS


@MODELS.register_module()
class MyAwesomeModel(BaseModel):
    def __init__(self, layers=4, activation='relu') -> None:
        super().__init__()
        if activation == 'relu':
            act_type = nn.ReLU
        elif activation == 'silu':
            act_type = nn.SiLU
        elif activation == 'none':
            act_type = nn.Identity
        else:
            raise NotImplementedError
        sequence = [nn.Linear(2, 64), act_type()]
        for _ in range(layers-1):
            sequence.extend([nn.Linear(64, 64), act_type()])
        self.mlp = nn.Sequential(*sequence)
        self.classifier = nn.Linear(64, 2)

    def forward(self, data, labels, mode):
        x = self.mlp(data)
        x = self.classifier(x)
        if mode == 'tensor':
            return x
        elif mode == 'predict':
            return F.softmax(x, dim=1), labels
        elif mode == 'loss':
            return {'loss': F.cross_entropy(x, labels)}


@DATASETS.register_module()
class MyDataset(Dataset):
    def __init__(self, is_train, size):
        self.is_train = is_train
        if self.is_train:
            torch.manual_seed(0)
            self.labels = torch.randint(0, 2, (size,))
        else:
            torch.manual_seed(3407)
            self.labels = torch.randint(0, 2, (size,))
        r = 3 * (self.labels+1) + torch.randn(self.labels.shape)
        theta = torch.rand(self.labels.shape) * 2 * torch.pi
        self.data = torch.vstack([r*torch.cos(theta), r*torch.sin(theta)]).T

    def __getitem__(self, index):
        return self.data[index], self.labels[index]

    def __len__(self):
        return len(self.data)


@METRICS.register_module()
class Accuracy(BaseMetric):
    def __init__(self):
        super().__init__()

    def process(self, data_batch, data_samples):
        score, gt = data_samples
        self.results.append({
            'batch_size': len(gt),
            'correct': (score.argmax(dim=1) == gt).sum().cpu(),
        })

    def compute_metrics(self, results):
        total_correct = sum(r['correct'] for r in results)
        total_size = sum(r['batch_size'] for r in results)
        return dict(accuracy=100*total_correct/total_size)

In [ ]:
from torch.utils.data import DataLoader, default_collate
from torch.optim import Adam
from mmengine.runner import Runner


runner = Runner(
    # 你的模型
    model=MyAwesomeModel(
        layers=2,
        activation='relu'),
    # 模型检查点、日志等都将存储在工作路径中
    work_dir='exp/my_awesome_model',

    # 训练所用数据
    train_dataloader=DataLoader(
        dataset=MyDataset(
            is_train=True,
            size=10000),
        shuffle=True,
        collate_fn=default_collate,
        batch_size=64,
        pin_memory=True,
        num_workers=2),
    # 训练相关配置
    train_cfg=dict(
        by_epoch=True,   # 根据 epoch 计数而非 iteration
        max_epochs=10,
        val_begin=2,     # 从第 2 个 epoch 开始验证
        val_interval=1), # 每隔 1 个 epoch 进行一次验证

    # 优化器封装，MMEngine 中的新概念，提供更丰富的优化选择。
    # 通常使用默认即可，可缺省。有特殊需求可查阅文档更换，如
    # 'AmpOptimWrapper' 开启混合精度训练
    optim_wrapper=dict(
        optimizer=dict(
            type=Adam,
            lr=0.001)),
    # 参数调度器，用于在训练中调整学习率/动量等参数
    param_scheduler=dict(
        type='MultiStepLR',
        by_epoch=True,
        milestones=[4, 8],
        gamma=0.1),

    # 验证所用数据
    val_dataloader=DataLoader(
        dataset=MyDataset(
            is_train=False,
            size=1000),
        shuffle=False,
        collate_fn=default_collate,
        batch_size=1000,
        pin_memory=True,
        num_workers=2),
    # 验证相关配置，通常为空即可
    val_cfg=dict(),
    # 验证指标与验证器封装，可自由实现与配置
    val_evaluator=dict(type=Accuracy),

    # 以下为其他进阶配置，无特殊需要时尽量缺省
    # 钩子属于进阶用法，如无特殊需要，尽量缺省
    default_hooks=dict(
        # 最常用的默认钩子，可修改保存 checkpoint 的间隔
        checkpoint=dict(type='CheckpointHook', interval=1)),

    # `luancher` 与 `env_cfg` 共同构成分布式训练环境配置
    launcher='none',
    env_cfg=dict(
        cudnn_benchmark=False,   # 是否使用 cudnn_benchmark
        backend='nccl',   # 分布式通信后端
        mp_cfg=dict(mp_start_method='fork')),  # 多进程设置
    log_level='INFO',

    # 加载权重的路径 (None 表示不加载)
    load_from=None,
    # 从加载的权重文件中恢复训练
    resume=False
)

# 开始训练你的模型吧
runner.train()